## Тестовое задание: Логинов Сергей на позицию «Data Analyst»

### Описание задания

#### 1 

Даны 3 витрины (daily_trx, merchant_info, cred_potential), которые содержат:  
•	daily_trx  – дневные данные по транзакциям клиента с 2018 года  
•	merchant_info – справочник точки совершения операции (продукты, спорт, АТМ)  
•	cred_potential -  данные о кредитном потенциале клиента (сумма кредитных средств, которую банк готов предложить клиенту)  


Необходимо:
1)	 По каждому клиенту найти кол-во дней (с начала 2019г. до конца июня 2019г.), которое ему потребовалось, чтобы совершить транзакций (расходных операций) на сумму не меньшую, чем его кредитный потенциал.   
Если он так и не потратил его, то вывести по клиенту значение = 200  
2)	Опишите логику решения задачи


#### 2 

Бизнес-кейс  
Заказчик просит выделить в клиентской базе Банка клиентов, которых можно привлечь в канал премиального обслуживания для ВИПов. Сейчас в канале 20 тыс. клиентов  - в основном наши зарплатные клиенты, клиенты с высокими балансами или крупными кредитами. Необходимо приземлить задачу на язык данных:
1)	Определить  признаки, которые помогут найти «латентных» ВИПов (минимум 3)  
2)	Определиться с подходом к решению задачи









# Решение Задачи 1

### Инструменты использованные в ходе выполнения задания: Python, DBeaver, SQLite3

# 1. Были созданы таблицы на основе данных из примера


In [ ]:

CREATE TABLE daily_trx (
    client_dk NUMBER PRIMARY KEY,      -- Идентификатор клиента
    Trx_id NUMBER NOT NULL,            -- Идентификатор транзакции
    Trx_dt DATE NOT NULL,              -- Дата транзакции
    Trx_amt NUMBER NOT NULL,           -- Сумма транзакции
    Trx_type VARCHAR2(1) NOT NULL,     -- Тип транзакции (C – расходная, D – приходная)
    Merch_id NUMBER                    -- Точка совершения операции (внешний ключ к merchant_info)
);

CREATE TABLE merchant_info (
    Merch_id NUMBER PRIMARY KEY,       -- Идентификатор точки совершения операции
    Mcc_code NUMBER NOT NULL,          -- Код типа магазина
    Mcc_name VARCHAR2(100) NOT NULL    -- Название типа магазина
);

CREATE TABLE cred_potential (
    client_dk NUMBER PRIMARY KEY,      -- Идентификатор клиента
    Crd_potential NUMBER NOT NULL      -- Сумма кредитных средств
);


INSERT INTO daily_trx (client_dk, Trx_id, Trx_dt, Trx_amt, Trx_type, Merch_id) VALUES
(507851886, 165745, TO_DATE('02.02.2019', 'DD.MM.YYYY'), 6400, 'C', 8234),
(267188214, 7821, TO_DATE('03.03.2019', 'DD.MM.YYYY'), 13200, 'C', 10454),
(148849526, 575631, TO_DATE('14.05.2019', 'DD.MM.YYYY'), 35000, 'D', 35475),
(613898474, 783123, TO_DATE('05.01.2019', 'DD.MM.YYYY'), 61000, 'C', 262343);

INSERT INTO merchant_info (Merch_id, Mcc_code, Mcc_name) VALUES
(8234, 15, 'Продукты'),
(10454, 2, 'Снятие наличных в банкомате'),
(35475, 33, 'Заработная плата'),
(262343, 12, 'Снятие наличных в кассе');

INSERT INTO cred_potential (client_dk, Crd_potential) VALUES
(507851886, 100000),
(267188214, 250000),
(148849526, 1000000),
(613898474, 50000);


# 2. Для обнаружения количества дней  по каждому клиенту задача была разбита на 3 этапа:

### - Шаг 1. Отсортировать транзакции по дате и клиенту. Для каждого клиента вычислить накопленную сумму расходных операций (только транзакции типа 'C')
### - Шаг 2. Когда накопленная сумма становится больше или равной кредитному потенциалу клиента (>=),  зафиксировать дату этого события. 
### - Шаг 3. Если накопленная сумма не достигает кредитного потенциала, вывести значение 200

   

## Шаг 1
1. В первом блоке кода мы выбираем необходимые данные таблицы daily_trx, а также добавляем агрегирующую оконную функцию, которая вычисляет кумулятивную сумму транзакций для каждого клиента.   
2. Присоединяем таблицу с датами (cred_potential) для фильтрации данных по дате.  
3. Фильтруем данные, где Trx_type - "С" - расходные транзакции, а также где даты между '2019-01-01' и '2019-06-30'.

In [ ]:
WITH cumulative_trx AS (
    SELECT
        dt.client_dk,
        dt.Trx_dt,
        dt.Trx_amt,
        SUM(dt.Trx_amt) OVER (PARTITION BY dt.client_dk ORDER BY dt.Trx_dt ROWS BETWEEN UNBOUNDED PRECEDING 
                              AND CURRENT ROW) AS cumulative_sum,
        cp.Crd_potential
    FROM
        daily_trx dt
    LEFT JOIN cred_potential cp ON cp.client_dk = dt.client_dk
    WHERE dt.Trx_type = 'C'
      AND dt.Trx_dt BETWEEN '2019-01-01' AND '2019-06-30'
),

## Шаг 2 
1. Во втором блоке кода мы находим самую раннюю дату совершения транзакции MIN(Trx_dt)
2. Далее мы фильтруем данные где наша накопленная сумма превышает кредитный потенциал клиента.


In [ ]:
first_exceed AS (
    SELECT
        client_dk,
        MIN(Trx_dt) AS first_exceed_date
    FROM
        cumulative_trx
    WHERE cumulative_sum >= Crd_potential
    GROUP BY client_dk
)

## Шаг 3

1. В последнем блоке мы выделяем id клиентов client_dk, 
2. Подтягиваем значения из предыдущей таблицы (также это позволяет выявить значения, по которым сумма не достигла крединтного потенциала из условия cumulative_sum >= Crd_potential. 
3. Вычисляем количество дней вычитая из даты достижения клиентом кредитного потенциала, в тех значениях, что возвращают NULL мы присваиваем 200. 

(!) В работе мной был использован SQLite3 для корректного запуска кода, синтаксис других СУБД в этой части кода отличается, любезно прошу вас учесть это при проверке теста. 



In [ ]:
SELECT
    c.client_dk,
    COALESCE(
        CAST(JULIANDAY(f.first_exceed_date) - JULIANDAY('2019-01-01') AS INTEGER),
        200
    ) AS days_to_exceed
FROM
    cred_potential c
LEFT JOIN first_exceed f ON f.client_dk = c.client_dk
ORDER BY c.client_dk;


## Код в общем виде:

In [ ]:
#Код в общем виде

WITH cumulative_trx AS (
    SELECT
        dt.client_dk,
        dt.Trx_dt,
        dt.Trx_amt,
        SUM(dt.Trx_amt) OVER (PARTITION BY dt.client_dk ORDER BY dt.Trx_dt ROWS BETWEEN UNBOUNDED PRECEDING 
                              AND CURRENT ROW) AS cumulative_sum,
        cp.Crd_potential
    FROM
        daily_trx dt
    LEFT JOIN cred_potential cp ON cp.client_dk = dt.client_dk
    WHERE dt.Trx_type = 'C'
      AND dt.Trx_dt BETWEEN '2019-01-01' AND '2019-06-30'
),
first_exceed AS (
    SELECT
        client_dk,
        MIN(Trx_dt) AS first_exceed_date
    FROM
        cumulative_trx
    WHERE cumulative_sum >= Crd_potential
    GROUP BY client_dk
)
SELECT
    c.client_dk,
    COALESCE(
        CAST(JULIANDAY(f.first_exceed_date) - JULIANDAY('2019-01-01') AS INTEGER),
        200
    ) AS days_to_exceed
FROM
    cred_potential c
LEFT JOIN first_exceed f ON f.client_dk = c.client_dk
ORDER BY c.client_dk;


________
_______

# Решение Задачи 2

1) Определить признаки, которые помогут найти «латентных» ВИПов (минимум 3)  
2) Определиться с подходом к решению задачи


## Три главных признака:

### Признак 1: Размер баланса на счетах

Краткое описание: Высокий баланс на счетах клиента может указывать на его финансовую состоятельность, что является важным фактором для премиального обслуживания.  
Метрика: Средний баланс на счете, минимальный баланс (например, больше 8 млн рублей)(источник: https://www.sravni.ru/text/vzyat-zajm-v-proverennoj-mfo/).  

### Признак 2: Использование кредитных продуктов

Краткое описание: Активность в использовании кредитных продуктов может свидетельствовать о более глубоком взаимодействии клиента с банком, что открывает возможности для предложения премиальных услуг.  
Метрика: Сумма активных кредитов, частота использования кредитных карт, объем погашенных кредитов.  

### Признак 3: Долгосрочная лояльность

Краткое описание: Клиенты, долгое время остающиеся с банком, могут быть заинтересованы в улучшении условий обслуживания и переходе на премиальный уровень.  
Метрика: Коэффициент удержания клиентов, продолжительность обслуживания (например, клиенты с опытом более 5 лет).  


## Подход к решению задачи

### Признак 1: Размер баланса на счетах
Данные: Используем информацию о текущих балансах клиентов на их расчетных счетах и депозитах. Собираем данные по остаткам на счетах за определенный период. Создаем сегментацию клиентов по величине остатка на их основном счете.
- Учитываем средний баланс на всех счетах клиента, чтобы оценить общую финансовую стабильность.
- Минимальный баланс за выбранный период (например, последний месяц).
- Регулярность транзакций — насколько часто происходят поступления и снятия, что может указывать на активность клиента.
- Качество (стабильность) поступлений.
- Долгосрочная динамика баланса — анализируем, растет ли баланс или снижается, чтобы понять, накапливает ли клиент средства, что может быть признаком стремления к улучшению финансового положения.


### Признак 2: Использование кредитных продуктов
Данные: Собираем данные о кредитных продуктах клиентов — это могут быть кредиты, кредитные карты, ипотека и прочие заемные средства. Выявляем, сколько и какие продукты активны у клиента, а также как часто они используются.
Для каждого клиента рассчитываем:
- Сумма всех активных кредитов.
- Частота использования кредитных продуктов (например, сколько раз в месяц используется кредитная карта).
- Статус по кредитам: есть ли задолженности, закрытые ли кредиты или текущие.
- Долгосрочность кредитной активности: есть ли кредиты, открытые более 1 года.  
Метрики: Средний объем задолженности по кредитам, коэффициент использования кредита (отношение долга к кредитному лимиту), количество активных кредитных продуктов. Также можно использовать данные по своевременному погашению (например, сколько месяцев клиент платил в срок).


### Признак 3: Долгосрочная лояльность
Данные: Для этого признака важны данные о длительности взаимоотношений клиента с банком. Необходимо знать дату открытия счета, дату получения первого кредита, а также частоту использования продуктов банка.  
Как анализируем: Для каждого клиента рассчитываем:  
- Срок от открытия счета до текущей даты.
- Частота посещений банка или использования онлайн-услуг (например, через мобильное приложение).
- Приверженность банку: сколько продуктов использует клиент (счета, кредиты, карты и т. д.).  
Метрики: Длительность обслуживания в банке, количество операций за последние 12 месяцев, коэффициент удержания (если клиент пользуется хотя бы одним продуктом банка больше 3 лет).

